Hace falta algo que indique con qué entorno vamos a trabajar

Importar lo que hace falta

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

In [2]:
data_url = "../Data/properatti.csv"

In [3]:
data = pd.read_csv(data_url, encoding="utf-8")

In [4]:
#limpiamos los que tienen NaN en el precio
data = data.dropna(axis=0, how='any', subset=['price_aprox_usd'])

In [5]:
#funcion para borrar outliers.
def borrar_outliers(data, columnas):
    """Solo recibo columnas con valores numericos. 
    Las columns van en forma de tupla"""
    cols_limpiar = columnas
    mask=np.ones(shape=(data.shape[0]), dtype=bool)

    for i in cols_limpiar:
        
        #calculamos cuartiles, y valores de corte
        Q1=data[i].quantile(0.25)
        Q3=data[i].quantile(0.75)
        RSI=Q3-Q1
        max_value=Q3+1.5*RSI
        min_value=Q1-1.5*RSI
        
        #ajusto el min value a mano... no puede ser negativo.
        min_value=max(min_value, data[i].quantile(0.05))
        
        #filtramos por max y min
        mask=np.logical_and(mask, np.logical_and(data[i]>=min_value, data[i]<=max_value))
    return data[mask]

In [6]:
# por ahora sacamos los outliers de precio y metros cubiertos de todo el dataset, habría que hacerlo por zona y tipo

data = borrar_outliers(data, ('price_aprox_usd', 'surface_covered_in_m2'))

## Categorizar precio

In [7]:
price_desc = data['price_aprox_usd'].describe()

In [8]:
price_desc

count     71285.000000
mean     178015.322455
std      105479.646428
min       53279.490000
25%       95237.990000
50%      145000.000000
75%      235000.000000
max      526957.000000
Name: price_aprox_usd, dtype: float64

In [9]:
price_bins = round( (price_desc['max'] - price_desc['min']) / 50000 ) 

In [10]:
price_cut = pd.cut(data['price_aprox_usd'], bins = price_bins, right = False, labels = None, retbins = False, precision = 0)

## Categorizar superficie cubierta

In [11]:
surface_desc = data['surface_covered_in_m2'].describe()

In [12]:
surface_desc

count    71285.000000
mean        95.697889
std         64.493952
min         29.000000
25%         47.000000
50%         71.000000
75%        127.000000
max        307.000000
Name: surface_covered_in_m2, dtype: float64

In [13]:
surface_bins = round( (surface_desc['max'] - surface_desc['min']) / 50 ) 

In [14]:
surface_cut = pd.cut(data['surface_covered_in_m2'], bins = surface_bins, right = False, retbins = False, precision = 0)

## tabla de contingencia

In [15]:
pd.crosstab(index = price_cut, columns = surface_cut, margins = True, dropna = True, normalize = 'columns')

surface_covered_in_m2,"[29.0, 75.0)","[75.0, 122.0)","[122.0, 168.0)","[168.0, 214.0)","[214.0, 261.0)","[261.0, 307.0)",All
price_aprox_usd,,,,,,,
"[53279.0, 105910.0)",0.487640,0.145705,0.071799,0.033408,0.014493,0.017484,0.302406
"[105910.0, 158541.0)",0.317237,0.262228,0.181272,0.088342,0.041017,0.038353,0.252325
"[158541.0, 211172.0)",0.119052,0.211383,0.234098,0.160840,0.098168,0.063170,0.150593
"[211172.0, 263803.0)",0.045334,0.160055,0.180317,0.184949,0.135630,0.106599,0.100063
"[263803.0, 316434.0)",0.013969,0.100034,0.127901,0.182711,0.154498,0.134236,0.067125
"[316434.0, 369064.0)",0.008293,0.057399,0.080945,0.163423,0.188953,0.172025,0.051722
"[369064.0, 421695.0)",0.004552,0.034012,0.057876,0.108145,0.166803,0.197406,0.037582
"[421695.0, 474326.0)",0.002302,0.016212,0.036582,0.047701,0.124419,0.143824,0.022136
"[474326.0, 527431.0)",0.001622,0.012970,0.029211,0.030480,0.076019,0.126904,0.016048


### Existe una importate asociación entre el precio en dólares y la superficie cubierta